# OpenFEMA API Tutorial: Part 2 - Query Parameters
  
## Quick Summary
- This tutorial demonstrates use of the OpenFEMA API query parameters - *it is not a Python language tutorial*
- The OpenFEMA API has a rich set of features that allow data to be filtered, sorted, and integrated into your own apps or automations
- An API parameter is a name or keyword that when added to a base query and endpoint will change the endpoint's behavior
- Several categories of OpenFEMA API parameters exist: metadata, data limiting, content altering, and data delivery
- A [Final Working Example](#Final-Working-Example) demonstrates the use and combination of many of the API parameters using Python
- The next tutorial will cover OpenFEMA API "paging" - retrieving more than 1,000 records in code using the \\$skip and \\$top parameters and additional usage examples


## Overview
In the previous tutorial we demonstrated the basic use of the OpenFEMA API - declaring a library to facilitate web requests, specifing the endpoint URL, issuing the OpenFEMA API call, and capturing the results. The OpenFEMA API provides many parameters that can be used to alter the nature of the data returned. Some parameters alter the content of the data - the fields to return, the sort order, and *what* records to return based on criteria. Other parameters affect data delivery or *how* the data is returned - file format, file names, and the number of records to return at one time. Finally, some parameters are used to alter the metadata.

The purpose of this notebook is to demonstrate the use of the most common OpenFEMA API parameters. Use of these parameters will significantly enhance your experience with the API. The examples are presented using Python 3, but it should be easy to translate them to almost any programming language.

For a summary and additional examples, please see the [OpenFEMA Documentation](https://www.fema.gov/about/openfema/api).

## What is an API parameter?
An API parameter is simply a name or keyword that when added to a base query and endpoint will change the endpoint's behavior. Parameters can be used for various reasons, such as to insert data into a system, as a switch mechanism to turn things on or off, or to provide instructions to the API to filter returned data based on a value.

API parameters are added to the end of the base URL and endpoint call.

![Sample API Call](img/anatomy_of_api_call.png)

### Anatomy of an API Query
At first glance, the query may seem complicated. Lets take a closer look at the composition of a query string and explain the various components:

![Sample Query String](img/anatomy_of_query_string.png)

- **Base URL/Query Separator** - The question mark (?) acts as a separator between the base URL and endpoint and the query or parameter list. Lack of this separator will cause the call to fail.
- **Parameter** - The parameter consists of a name, and for OpenFEMA is prefixed with a dollar sign to clearly make a distinction between the parameter and the value that follows. A full list of the parameters offered by OpenFEMA can be found in the [OpenFEMA Documentation](https://www.fema.gov/about/openfema/api).
- **Parameter Name/Value Separator** - A parameter *value* must follow the parameter name. The equals sign (=) separates these two.
- **Value** - The instruction or value that is assigned to the parameter. This will vary depending on the context of the parameter.
- **URL Encoded Character** - Some characters have special meaning within a URL such as a pound sign (#) or a forward slash (/). Some characters while permitted, such as spaces, can be ambiguous. URL Encoding is used to deal with these problems. Codes are used to represent problematic characters. For example, %20 represents a space and %27 represents a single quote. A web search of "url encoding" or "percent encoding" will produce a list of the code values for problematic characters.
- **Logical Operator** - Optional for many parameters. Some parameters, such as \\$filter, need a logical operator for comparison purposes. The equals sign (=) cannot be used as it is currently in use as the Parameter Name/Value Separator. OpenFEMA uses textual values such as "eq" for "equal to" and "gt" for "greater than". See the documentation for a list of available operators.
- **Parameter Pair Separator** - An API query can use many parameters together. To separate the parameters, the ampersand sign (&) is used. This tells the OpenFEMA API that the previous parameter is terminating and another follows.
- **Next Parameter/Value Pair** - Any number of parameters may be used in the same query. **The order in which parameters are combined is not important.**

For more information, see the [Wikipedia Query String page](https://en.wikipedia.org/wiki/Query_string)

<div class="alert alert-block alert-info">
    <b>Tip:</b> Do not add spaces between the the URL/Query Separator (?), the Parameter name including the dollar sign, the Name/Value separator (=), and the Parameter Pair Separator (&) otherwise the call will fail.  
</div>

<div class="alert alert-block alert-warning">
    <b>Note:</b> There can be slight variation in the notation and symbols used across different web APIs. The notation and examples used here apply to the OpenFEMA API.
</div>

### Why would I want to use API parameters rather than manipulate the full dataset myself?
It is true that you can download the full dataset and filter/transform it in a spreadsheet, using code, or your favorite  analytic tool. However, there are a few benefits of taking advantage of the OpenFEMA API parameters:
 - Reducing the amount of data you need to download. This can reduce download times and eliminate work to filter/transform. This is especially relevant when dealing with very large datasets.
 - Offload computer processing to someone else's server - OpenFEMA's!
 - To return data in a format that makes it easier for you to process or ingest.
 
## Calling an OpenFEMA Endpoint with a Parameter
Assuming you have identified an endpoint (see [OpenFEMA DataSets](https://www.fema.gov/about/openfema/data-sets)), determine the parameters you wish to apply (we will discuss them later), and add them to the end of the base URL/endpoint - separated with the question mark of course. The resulting URL can be accessed by a browser, by code, or through many third-party analytics applications.

As explained in part 1 of the tutorial, the process to call an API endpoint is straightforward - declare a library or module that can facilitate requests to web resources, define the URL/API endpoint - now including the parameters (aka, the query, the request) and issue the API call.

The following example will combine the base url and endpoint with the parameter.


In [31]:
# declare a URL handling module
import urllib.request
import json

# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations"

# define a query - use a parameter to return only the first 2 records
query = "?$top=2"

print(baseUrl + query)


https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations?$top=2


Now lets execute the request, decode and save the result in a variable, and print the results. As explained in part 1, by default the OpenFEMA API will return data in a JSON format and include a metadata object. Later we will demonstrate the use of parameters to alter this behavior.

In [32]:
# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + query)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData, indent=2))

{
  "metadata": {
    "skip": 0,
    "filter": "",
    "orderby": "",
    "select": null,
    "rundate": "2022-10-04T21:26:09.077Z",
    "entityname": "FemaWebDisasterDeclarations",
    "version": "v1",
    "top": 2,
    "count": 0,
    "format": "json",
    "metadata": true,
    "url": "/api/open/v1/FemaWebDisasterDeclarations?$top=2"
  },
  "FemaWebDisasterDeclarations": [
    {
      "disasterNumber": 4659,
      "declarationDate": "2022-07-13T00:00:00.000Z",
      "disasterName": "SEVERE STORMS, STRAIGHT-LINE WINDS, AND FLOODING",
      "incidentBeginDate": "2022-04-22T00:00:00.000Z",
      "incidentEndDate": "2022-06-15T00:00:00.000Z",
      "declarationType": "Major Disaster",
      "stateCode": "MN",
      "stateName": "Minnesota",
      "incidentType": "Flood",
      "entryDate": "2022-07-14T00:00:00.000Z",
      "updateDate": "2022-07-14T00:00:00.000Z",
      "closeoutDate": null,
      "hash": "4249872e48eaecbac8ef0807463f2c82fa183695",
      "id": "be71ef8f-2efb-49f1-8c60-39

## Metadata Parameters
While it would seem like beginning with the parameters to affect content would be a more intuitive place to start, there are a couple of important metadata parameters that will make our lives much easier if we learn about them now.

### \\$inlinecount
An option that controls if a total count of all entities matching the request *must* be returned as part of the result. 

Understanding this parameter is easier with an example. We will execute the same query as above without the \\$top parameter to limit the records returned. Further, we will only show the metadata object.

In [5]:
# declare a URL handling module
import urllib.request
import json

# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations"

# open the URL and create a the request object 
request = urllib.request.urlopen(baseUrl)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData['metadata'], indent=2))

{
  "skip": 0,
  "filter": "",
  "orderby": "",
  "select": null,
  "rundate": "2022-10-04T01:08:37.658Z",
  "entityname": "FemaWebDisasterDeclarations",
  "version": "v1",
  "top": 1000,
  "count": 0,
  "format": "json",
  "metadata": true,
  "url": "/api/open/v1/FemaWebDisasterDeclarations"
}


Take note of the "top" and "count" elements within this object. Top is indicating that a *maximum* of 1,000 will be returned. Count displays 0. How many records were returned? How many met our criteria if more than 1,000 records were *found*? **If the \\$inlinecount parameter is not supplied, no count will be returned; it will appear as “0”.**

To answer the first question "How many records were returned", we can count what was returned in Python (as the following example shows), but it does not answer the second - "how many were found".

In [6]:
# count the number of records in the dataset object using the length function
len(jsonData['FemaWebDisasterDeclarations'])

1000

The \\$inlinecount parameter accepts two possible values: 

 - allpages - will return a count of the records found
 - none - is the default value and will not return a count

Let's execute the same request, but with the \\$inlinecount parameter set. We will also count the records returned in Python.

In [7]:
# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations"

# define our parameters
parameters = "?$inlinecount=allpages"

# open the URL combined with the parameters and create a the request object 
request = urllib.request.urlopen(baseUrl + parameters)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData['metadata'], indent=2))
len(jsonData['FemaWebDisasterDeclarations'])

{
  "skip": 0,
  "filter": "",
  "orderby": "",
  "select": null,
  "rundate": "2022-10-04T01:10:05.820Z",
  "entityname": "FemaWebDisasterDeclarations",
  "version": "v1",
  "top": 1000,
  "count": 4713,
  "format": "json",
  "metadata": true,
  "url": "/api/open/v1/FemaWebDisasterDeclarations?$inlinecount=allpages"
}


1000

The count element indicates more than 4,700 records were found (as we have no criteria, this represents a count of the full dataset). Since a maximum of 1,000 are returned per call, only the first 1,000 have been returned (as confirmed by the Python count). Retrieving all the records requires a technique called "paging" that utilizes the \\$skip parameter, \\$top parameter, and the "count" value to make several calls. This will be covered in detail in a future tutorial.

<div class="alert alert-block alert-warning">
    <b>Note:</b> If retrieving large amounts of data or using complex \$filter criteria through a "paging" technique, avoid specifying "allpages" in each call. This will result in faster queries.
</div>



### \\$metadata
Controls the presence of the metadata object with a returned dataset. This parameter only applies if the output format is JSON. If the format is CSV, no metadata is returned.

This parameter can be especially useful if you are automating a process (such as paging to retrieve all the data) and do not want or need the metadata object. Turning metadata off (i.e., \\$metadata=off) can speed up large downloads by reducing the amount of data returned as well as eliminating the need for the OpenFEMA server to generate the object.

Supported values are:

- on – Returns the metadata object for data returned in a JSON format (default)
- off – Suppresses the metadata object for data returned in a JSON format

## Data Limiting Parameters
The data limiting we will discuss here is not related to *filtering* the data - we will discuss that later. It refers to limiting the amount of data returned irrespective of any filters applied. Limiting data is done by calculating an offset (or starting point) and specifying the number of records to return. These two parameters are often used with the "count" element in the metadata to iterate through and capture a full dataset. 

### \\$top
Limits the number of records returned. Currently the default and maximum value is 1,000 records. When first evaluating a dataset it is often useful to initially review a few records.

<div class="alert alert-block alert-info">
    <b>Tip:</b> Sometimes, all you may need is a count matching some set of criteria. The following example uses the \\$inlinecount and \\$top parameters to return just 1 record, where the count is extracted from the metadata object.
</div>

In [8]:
# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations"

# define our parameters - return a count and limit to 1 record
parameters = "?$inlinecount=allpages&$top=1"

# open the URL combined with the parameters and create a the request object 
request = urllib.request.urlopen(baseUrl + parameters)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData['metadata']['count'], indent=2))

4713


### \\$skip
Number of records to skip from the dataset. Used in conjunction with \\$top to allow you to page through the dataset. If no value is specified, \\$skip defaults to 0 and starts at the beginning of the dataset. The following example will retrieve two records - skipping the first 10.

In [9]:
# define our parameters - return a count and limit to 1 record
parameters = "?$skip=10&$top=2"

# open the URL combined with the parameters and create a the request object 
request = urllib.request.urlopen(baseUrl + parameters)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData, indent=2))

{
  "metadata": {
    "skip": 10,
    "filter": "",
    "orderby": "",
    "select": null,
    "rundate": "2022-10-04T01:15:42.674Z",
    "entityname": "FemaWebDisasterDeclarations",
    "version": "v1",
    "top": 2,
    "count": 0,
    "format": "json",
    "metadata": true,
    "url": "/api/open/v1/FemaWebDisasterDeclarations?$skip=10&$top=2"
  },
  "FemaWebDisasterDeclarations": [
    {
      "disasterNumber": 4657,
      "declarationDate": "2022-06-29T00:00:00.000Z",
      "disasterName": "SEVERE STORMS, TORNADOES, AND FLOODING",
      "incidentBeginDate": "2022-05-02T00:00:00.000Z",
      "incidentEndDate": "2022-05-08T00:00:00.000Z",
      "declarationType": "Major Disaster",
      "stateCode": "OK",
      "stateName": "Oklahoma",
      "incidentType": "Severe Storm(s)",
      "entryDate": "2022-06-30T00:00:00.000Z",
      "updateDate": "2022-07-07T00:00:00.000Z",
      "closeoutDate": null,
      "hash": "8b6164f92901ffd0c8c0125098f127cb2a70924c",
      "id": "fd91a0d1-ee4b-4b9

<div class="alert alert-block alert-warning">
    <b>Note:</b> It is dangerous to use \$skip and \$top without specifying a sort order (\$orderby discussed below). Although extremely rare, there is no guarantee that the data will be returned in the same order in subsequent calls. If you do not care about the order, use the ID field as the field to sort. 
</div>

## Content Altering Parameters 
The previous example displays the full disaster summary record (for every record if we remove $top), in no particular order. Being able retrieve only the data of interest to you in an order you want is very desirable.

### \\$select
Used to specify which fields you would like returned. Providing a comma separated list of *case sensitive* field names will return just those fields (see the specific dataset pages for field names). If no value is specified, all the fields are returned.

Using the previous example, lets return just the disaster number, declaration date, name, and state.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Within Python, adding spaces in the query string will cause an error. This is not the case if you execute the query from a browser. 
</div>

In [10]:
# define a query - use a parameter to select only specific fields
query = "?$select=disasterNumber,declarationDate,disasterName,stateCode"

# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + query)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# print only the first 3 from the dataset object only (this can also be done with $top)
print(json.dumps(jsonData['FemaWebDisasterDeclarations'][0:3], indent=2))

[
  {
    "disasterNumber": 4659,
    "declarationDate": "2022-07-13T00:00:00.000Z",
    "disasterName": "SEVERE STORMS, STRAIGHT-LINE WINDS, AND FLOODING",
    "stateCode": "MN"
  },
  {
    "disasterNumber": 5441,
    "declarationDate": "2022-06-12T00:00:00.000Z",
    "disasterName": "PIPELINE FIRE",
    "stateCode": "AZ"
  },
  {
    "disasterNumber": 5444,
    "declarationDate": "2022-07-19T00:00:00.000Z",
    "disasterName": "CHALK MOUNTAIN FIRE",
    "stateCode": "TX"
  }
]


### \\$orderby
While the data returned may look like it is in some natural order, it cannot be guaranteed. Use the \\$orderby parameter if the order in which the returned data is sorted is important to you. By providing a comma separated list of fields and a sort direction you can control the order that data is returned. Available sort directions are “asc” and “desc” for ascending or descending respectively. If no direction is provided, ascending is the default.

Building on the previous example, lets demonstrate combining parameters (note the ampersand separating the parameters) and sorting the returned data by the newest declaration date to the oldest.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Because the $orderby parameter requires a space separator between the sort field and the sort direction, and because Python will produce an error for a control character embedded in the URL, we must use the URL code of %20 instead of a space.
</div>

In [11]:
# define a query - use parameters to return several fields, newest declaration date to oldest
query = "?$select=disasterNumber,declarationDate,disasterName,stateCode&$orderby=declarationDate%20desc"

# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + query)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# print only the first 3 from the dataset object only ($top can also be used)
print(json.dumps(jsonData['FemaWebDisasterDeclarations'][0:3], indent=2))

[
  {
    "disasterNumber": 3586,
    "declarationDate": "2022-10-01T00:00:00.000Z",
    "disasterName": "HURRICANE IAN",
    "stateCode": "NC"
  },
  {
    "disasterNumber": 4675,
    "declarationDate": "2022-09-30T00:00:00.000Z",
    "disasterName": "HURRICANE IAN - SEMINOLE TRIBE OF FLORIDA",
    "stateCode": "FL"
  },
  {
    "disasterNumber": 4674,
    "declarationDate": "2022-09-30T00:00:00.000Z",
    "disasterName": "FLOODING AND MUDSLIDES",
    "stateCode": "VA"
  }
]


### \\$filter
The \\$filter parameter is used to limit or select specific data from the dataset. It is the most complex of the OpenFEMA API parameters. It permits multiple conditions to be chained together, it supports grouping, has many possible logical operators, and supports several special functions with which to enhance searches. This tutorial will focus on the basics - simple searches and compound searches. A future tutorial will highlight the special functions and more complex conditions including  hierarchical object searching.

See the [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api#filter) for additional details on the \\$filter parameter.

<div class="alert alert-block alert-info">
    <b>Tip:</b> The query and \$filter processor is very strict. Ensure your spacing, quoting, and capitalization are correct or the call will fail.<br>
    <ul>
    <li>Dataset field names are case sensitive.</li>
    <li>Be mindful of the field data type. Strings need to be single quoted, numbers do not, Booleans are expressed as true or false without quotes.</li>
    <li>Dates should be represented in an ISO-8601 format.</li>
    <li>Spaces are required between logical operators.</li>
    <li>URL encoding (e.g., %20, %24, %27, etc.) should be used in creating a Python URL string, but is optional in most browsers.</li>
    <li>Any string containing a quote (e.g., bob's burgers) must have the quote doubled to have an effect (e.g., bob''s burgers).</li>
    </ul>
</div>

Building on the previous example, lets demonstrate limiting returned data to disasters in the state of Virginia. Because the query string is starting to become unwieldly, we have stored each component in a separate variable. 

<div class="alert alert-block alert-warning">
    <b>Note:</b> An alternative to using separate variables would be to use the line continuation character (\) or the triple quote (""") for a multiline string, but this will introduce unwanted whitespace in the string if you choose to indent. Separate quoted strings wrapped in parenthesis will also work.
</div>


In [12]:
# define a query - use parameters to return several fields, sort, and pick only VA disasters
select = "?$select=disasterNumber,declarationDate,disasterName,stateCode"
filter = "&$filter=stateCode%20eq%20%27VA%27"
orderby = "&$orderby=declarationDate%20desc"


# lets display the full URL - order of concatenation is important as we embedded the ampersand 
#    in all but the first variable above, and added the Base URL/Query Separator to the first.
print(baseUrl + select + filter + orderby)

https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations?$select=disasterNumber,declarationDate,disasterName,stateCode&$filter=stateCode%20eq%20%27VA%27&$orderby=declarationDate%20desc


In [13]:
# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + select + filter + orderby)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# print only the first 3 from the dataset object only
print(json.dumps(jsonData['FemaWebDisasterDeclarations'][0:3], indent=2))

[
  {
    "disasterNumber": 4674,
    "declarationDate": "2022-09-30T00:00:00.000Z",
    "disasterName": "FLOODING AND MUDSLIDES",
    "stateCode": "VA"
  },
  {
    "disasterNumber": 4644,
    "declarationDate": "2022-03-11T00:00:00.000Z",
    "disasterName": "SEVERE WINTER STORM AND SNOWSTORM",
    "stateCode": "VA"
  },
  {
    "disasterNumber": 4628,
    "declarationDate": "2021-10-26T00:00:00.000Z",
    "disasterName": "FLOODING, LANDSLIDES, AND MUDSLIDES",
    "stateCode": "VA"
  }
]


## Parameters Affecting the Delivery of Data
A couple of parameters exist to define the file type or format and an optional filename.

### \\$format
By default, the API returns data in a json format. As we have seen in our examples, Python (and other programming languages) have an easy time manipulating json data, which is human readable as well. The \\$format parameter provides a mechanism to return data in other formats. Currently supported values include:

 - json - Returns data in the JavaScript Object Notation format (default).
 - jsona - Returns data as a JavaScript Object Notation array format. There is no top-level object and each object is separated by a comma. The metadata object is automatically suppressed if this format is chosen. 
 - geojson - Returns data in a special json format designed to represent geographical features. This format is only available for datasets that support it, such as the FemaRegions endpoint.
 - csv - Returns data in a Comma-Separated Value format. The metadata object is automatically suppressed if this format is chosen.
 
In the following example we will return the first 3 records in a jsona format. Note the metadata has been suppressed automatically. We have printed the combined URL for reference.

In [14]:
# define a query - use parameters to return several fields, sort, pick only the first 3 VA disasters, and deliver as jsona
select = "?$select=disasterNumber,declarationDate,disasterName,stateCode"
filter = "&$filter=stateCode%20eq%20%27VA%27"
orderby = "&$orderby=declarationDate%20desc"
limit = "&$top=3"
format = "&$format=jsona"

# lets display the full URL - order of concatenation is important as we embedded the ampersand 
#    in all but the first variable above, and added the Base URL/Query Separator to the first.
print(baseUrl + select + filter + orderby + limit + format)

# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

print(json.dumps(jsonData, indent=2))

https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations?$select=disasterNumber,declarationDate,disasterName,stateCode&$filter=stateCode%20eq%20%27VA%27&$orderby=declarationDate%20desc&$top=3&$format=jsona
[
  {
    "disasterNumber": 4674,
    "declarationDate": "2022-09-30T00:00:00.000Z",
    "disasterName": "FLOODING AND MUDSLIDES",
    "stateCode": "VA"
  },
  {
    "disasterNumber": 4644,
    "declarationDate": "2022-03-11T00:00:00.000Z",
    "disasterName": "SEVERE WINTER STORM AND SNOWSTORM",
    "stateCode": "VA"
  },
  {
    "disasterNumber": 4628,
    "declarationDate": "2021-10-26T00:00:00.000Z",
    "disasterName": "FLOODING, LANDSLIDES, AND MUDSLIDES",
    "stateCode": "VA"
  }
]


The following example returns data in a csv format. 

In [15]:
# define a query - use parameters to return several fields, sort, pick only the first 3 VA disasters, and deliver as csv
select = "?$select=disasterNumber,declarationDate,disasterName,stateCode"
filter = "&$filter=stateCode%20eq%20%27VA%27"
orderby = "&$orderby=declarationDate%20desc"
limit = "&$top=3"
format = "&$format=csv"

# open the URL combined with the query and create a the request object 
request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format)

# actually read the data
result = request.read()

# decode the data
csvData = result.decode('utf-8')

print(csvData)

disasterNumber,declarationDate,disasterName,stateCode
4674,2022-09-30T00:00:00.000Z,FLOODING AND MUDSLIDES,VA
4644,2022-03-11T00:00:00.000Z,SEVERE WINTER STORM AND SNOWSTORM,VA
4628,2021-10-26T00:00:00.000Z,"FLOODING, LANDSLIDES, AND MUDSLIDES",VA



The data above is a string containing comma separated value data and is not very easy to use at the moment. We can convert it to a Python dictionary by using the "csv" library. The most common "quick-and-dirty" method is to save the data as a csv file, then re-open it using the csv.DictReader function (shown).

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python class, but an alternative that does not first save the data to a file file or use any special library involves parsing each row based on the newline line terminator to create a "list" of lines, split each line to create a "table" (essentially a two dimensional array of rows and fields), and iterate through the array creating a "dictionary".
</div>

In [28]:
# define the csv library
import csv

# transform to Python dictionary
csvData = result.decode('utf-8')

# save data
with open("fwdd_file.csv", "w") as fp1:
    fp1.write(csvData)
    
# define a dictionary variable to hold our data
dictData = []

# open file and read using csv library DictReader 
with open('fwdd_file.csv', "r") as fp1:
    csvReader = csv.DictReader(fp1)
    
    # add each row of our data (now a dictionary) to our list
    for row in csvReader:
        dictData.append(row)
    
# now we have a data structure that we can work with as if we retrieved json data
print(json.dumps(dictData, indent=2))

[
  {
    "disasterNumber": "4674",
    "declarationDate": "2022-09-30T00:00:00.000Z",
    "disasterName": "FLOODING AND MUDSLIDES",
    "stateCode": "VA"
  },
  {
    "disasterNumber": "4644",
    "declarationDate": "2022-03-11T00:00:00.000Z",
    "disasterName": "SEVERE WINTER STORM AND SNOWSTORM",
    "stateCode": "VA"
  },
  {
    "disasterNumber": "4628",
    "declarationDate": "2021-10-26T00:00:00.000Z",
    "disasterName": "FLOODING, LANDSLIDES, AND MUDSLIDES",
    "stateCode": "VA"
  }
]


<div class="alert alert-block alert-info">
    <b>Tip:</b> Don't do this kind of conversion if you need data in a format conducive to analysis in code, just request the data in a json format and forget the conversion piece. If you need a csv format for analysis in a spreadsheet or for an import into a database, just save the returned csv data to a file.<br>
</div>

### \\$filename
Allows for the specification of a download filename. The data downloaded will be limited to fields specified using \\$select (all fields will be saved if none specified), will contain records limited by \\$filter, and will be sorted according to the \\$orderby parameter. The filename should not be surrounded by quotation marks even if the filename contains spaces. If no \\$format parameter is specified, JSON will be the result.

The \\$filename parameter is only relevant when executing a query from a browser. It will trigger the "download/file save" feature of the browser and save the data using the provided filename. Continuing from the examples above, execute the following query in a browser to save Virginia disaster declarations as a csv file to the filename given:

    https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations?$select=disasterNumber,declarationDate,disasterName,stateCode&$filter=stateCode%20eq%20%27VA%27&$orderby=declarationDate%20desc&$top=3&$format=csv&$filename=fwdd_test.csv


<div class="alert alert-block alert-warning">
    <b>Note:</b> The metadata object is automatically suppressed when using this parameter.
</div>

## Final Working Example
Although contrived, this final example applies many of the parameters discussed in this tutorial.  Examples in other languages can be found in the [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples) repository.

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python language tutorial. The point is to show the use and combination of OpenFEMA API parameters. There are other, more Pythonic ways that this can be done. If you are writing production quality code, it is recommended that you follow industry best practices - evaluate returned values, add error handling, add logging, proper object cleanup, build for resilience by adding retries if failure, etc.
</div>

In [30]:
# declare a URL handling module
import urllib.request
import json

# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,disasterName,stateCode"
filter = "&$filter=stateCode%20eq%20%27VA%27"
orderby = "&$orderby=declarationDate%20desc"
limit = "&$skip=10&$top=3"
format = "&$format=json"
other = "&$inlinecount=allpages"

# open the URL as defined above and create a the request object 
request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other)

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# POSSIBLY DO A BUNCH OF STUFF HERE

print(json.dumps(jsonData, indent=2))

{
  "metadata": {
    "skip": 10,
    "filter": "stateCode eq 'VA'",
    "orderby": "declarationDate DESC",
    "select": null,
    "rundate": "2022-10-04T15:55:07.820Z",
    "entityname": "FemaWebDisasterDeclarations",
    "version": "v1",
    "top": 3,
    "count": 73,
    "format": "json",
    "metadata": true,
    "url": "/api/open/v1/FemaWebDisasterDeclarations?$select=disasterNumber,declarationDate,disasterName,stateCode&$filter=stateCode%20eq%20%27VA%27&$orderby=declarationDate%20desc&$skip=10&$top=3&$format=json&$inlinecount=allpages"
  },
  "FemaWebDisasterDeclarations": [
    {
      "disasterNumber": 4262,
      "declarationDate": "2016-03-07T00:00:00.000Z",
      "disasterName": "SEVERE WINTER STORM AND SNOWSTORM",
      "stateCode": "VA"
    },
    {
      "disasterNumber": 4092,
      "declarationDate": "2012-11-26T00:00:00.000Z",
      "disasterName": "HURRICANE SANDY",
      "stateCode": "VA"
    },
    {
      "disasterNumber": 3359,
      "declarationDate": "2012-10-2

### Where to go Next
OpenFEMA API Tutorial Part_2_API_Paging. This next tutorial will discuss retrieving more than 1,000 records in code using the \\$top and \\$skip parameters.

## Other Resources
- [OpenFEMA Homepage](https://www.fema.gov/open)
- [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api)
- [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples)
- [ODATA URL Conventions](https://www.odata.org/documentation/odata-version-3-0/url-conventions/)